# SQL query from table names - Continued

In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [4]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [17]:
context = [ {'role':'system', 'content':"""
CREATE Table Bus Plan (
    BUS_ID INTEGER PRIMARY KEY,
    Route_Name TEXT,
    Departure_place TEXT,
    Arrival_place TEXT,
    Departure_time TEXT,
    Arrival_time TEXT,
    Driver_Name TEXT,
    Salary REAL
    );
             
CREATE Table Drivers (
    Driver_ID INTEGER PRIMARY KEY,
    Driver_Name TEXT,
    License_type TEXT,
    Experience_Years INTEGER, 
    Salary REAL
    );
             
CREATE Table Routes(
    Route_ID INTEGER PRIMARY KEY,
    Route_Name TEXT,
    Distance_KM REAL
);
"""} ]



In [18]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
-- Here are some examples:

Q: What is the highest driver salary?
A: SELECT MAX(Salary) AS highest salary
From Drivers;
                 
Q: Which driver has the highest salary?
A: SELECT Driver_Name
FROM Drivers
WHERE Salary=(SELECT MAX(Salary) FROM Drivers);
                 
Q: List all routes with their total distance.
A: SELECT Route_Name, Distance_KM
FROM Routes;
                 
Q: Which buses start from "Station X"?
A: SELECT Route_Name, Departure_Time
FROM Bus_Pan
WHERE Departure_Place= "Station A";                 
"""})

In [19]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [20]:
#new
context_user = context.copy()
print(return_CCRMSQL("Which route is the longest in kilometers?", context_user))

```sql
SELECT Route_Name
FROM Routes
WHERE Distance_KM = (SELECT MAX(Distance_KM) FROM Routes);
```


In [22]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Which route is the longest in kilometers?", old_context_user))

Nice question! Can you please provide the table structure or data related to routes?


In [24]:
#new
print(return_CCRMSQL("Which driver is taking Route C on Tuesday?", context_user))

SELECT Driver_Name
FROM Bus_Plan
WHERE Route_Name = 'Route C' AND Departure_time LIKE '%Tuesday%';


In [26]:
#old
print(return_CCRMSQL("Which driver is taking Route C on Tuesday", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN driver_routes dr ON e.ID_usr = dr.ID_usr
JOIN routes r ON dr.route_id = r.route_id
WHERE r.route_name = 'Route C' AND r.day = 'Tuesday';
```

This SQL query retrieves the name of the driver who is assigned to Route C on Tuesday. It joins the employees table with the driver_routes and routes tables to find the driver's name based on the route name and day.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [ ]:
#new
context_user = context.copy()
print(return_CCRMSQL("Which route is the longest in kilometers?", context_user))

```sql
SELECT Route_Name
FROM Routes
WHERE Distance_KM = (SELECT MAX(Distance_KM) FROM Routes);
```


In [ ]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Which route is the longest in kilometers?", old_context_user))

In [ ]:
Nice question! Can you please provide the table structure or data related to routes?

In [ ]:
#new
print(return_CCRMSQL("Which driver is taking Route C on Tuesday?", context_user))

SELECT Driver_Name
FROM Bus_Plan
WHERE Route_Name = 'Route C' AND Departure_time LIKE '%Tuesday%';

In [ ]:
#old
print(return_CCRMSQL("Which driver is taking Route C on Tuesday", old_context_user))

In [ ]:
This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN driver_routes dr ON e.ID_usr = dr.ID_usr
JOIN routes r ON dr.route_id = r.route_id
WHERE r.route_name = 'Route C' AND r.day = 'Tuesday';
```
This SQL query retrieves the name of the driver who is assigned to Route C on Tuesday. It joins the employees table with the driver_routes and routes tables to find the driver's name based on the route name and day.

To see the different anwers in the old and new prompt are really interesting. The way and quality of answer seems smarter and with a clearer context.
The old prompt had no schema. It didnt seem to know that tables existed, so that GPT had to guess what kind of tables might be available. Could very much likely come to hallucination I think because it lacked context.
So in general, I learned that zero shot prompting  has no schema and a high chance of being made up and in the few shot promting, it seemed clearer and has explicit schema.
The paper from Ohio stated the same thing, the few shot is to eliminate the hallucination. 
So, what is the zero shot really for? Is it only for very simple and small tables?